In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import gc
from pathlib import Path

import os
import os.path
import glob
import cv2

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import Sequential, Input, initializers, optimizers, callbacks
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Flatten, ReLU, PReLU
from tensorflow.keras.metrics import AUC
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten, GlobalAveragePooling2D, Activation
from tensorflow.keras.layers import AveragePooling2D, GlobalMaxPooling2D, Lambda
from tensorflow.keras.optimizers import Nadam, Adam

seed = 47

In [ ]:
import pandas as pd
import seaborn as sns

data = pd.read_csv('/kaggle/input/clothing-dataset-full/images.csv')
data.loc[data['label']=='Not sure','label'] = 'Not_sure'
#data['image'] = data['image'] + '.jpg'

labels=data['label']
features=data.drop('label',1)

labels_values=labels.value_counts()
print(labels_values)

# data['label_cat'] = data['label'] + ' ' + data['kids'].astype(str)
# proc_data=data[['image', 'label_cat']]
# print(type(proc_data))

p= sns.countplot(data=data['image'], x=data['label'])

In [ ]:
labels_y=data.label
labels_num1=labels_y[:1000]
labels_num1

In [ ]:
labels_unique=labels_num1.unique()
print(labels_unique.shape)
# data.loc[(data.label) & (data.image == '4285fab0-751a-4b74-8e9b-43af05deee22')]

In [ ]:
# foods = list(os.walk('../input/food41/images/'))[0][1]
nr_foods = 20
labels_values=list(labels_values)

idx_to_name = {i:x for (i,x) in enumerate(labels_unique[:nr_foods])}
name_to_idx = {x:i for (i,x) in enumerate(labels_unique[:nr_foods])}

idx_to_name

In [ ]:
import cv2
from PIL import Image
from tqdm import tqdm 

data_l = []
img_size = (64, 64)

for img in tqdm(data['image'][:1000]):
    imgs = [cv2.resize(cv2.imread(f'/kaggle/input/clothing-dataset-full/images_original/{img}.jpg'), img_size, interpolation=cv2.INTER_AREA)]
    for i in imgs:
        data_l.append(i)

In [ ]:
labels_y=[]

import numpy as np
for i in labels_num1:
    labels_y.append(name_to_idx[i])
len(labels_y)

In [ ]:
labels_y_s=pd.Series(labels_y)
type(labels_y_s)

In [ ]:
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder()
enc.fit(labels_y_s.values.reshape(-1, 1))

y_transformed = enc.transform(labels_y_s.values.reshape(-1, 1))

import scipy
y_one_hot=scipy.sparse.csr_matrix.toarray(y_transformed)

y_one_hot.shape

In [ ]:
data = np.array(data_l)
data = data / 255.0
data = data.astype('float32')

In [ ]:
labels_y_s[:1000].mean()

In [ ]:
X_train = data[:500]
X_test = data[500:]

y_train = labels_y[:500]
y_test = labels_y[500:]

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(data_l_3, y_one_hot,
#                                                     test_size=0.2,
#                                                     random_state=1,
#                                                     shuffle=True)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

class Sampling(layers.Layer):

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
latent_dim = 512

encoder_inputs = keras.Input(shape=(64, 64, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
tf.keras.layers.Dropout(0.2)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
tf.keras.layers.Dropout(0.2)
# x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
# x = layers.Dense(512, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(16 * 16 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((16, 16, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = (reconstruction_loss + kl_loss)/2000
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [ ]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
history = vae.fit(X_train, epochs=15, batch_size=32, verbose=1)

In [ ]:
# 2386 2333 2299 2290 2.34
preds_enc = vae.encoder.predict(X_train)
preds_enc[0].shape

In [ ]:
preds_dec = vae.decoder.predict(preds_enc[2])
preds_dec.shape

In [ ]:
print(history.history.keys())
plt.plot(history.history['loss'])

In [ ]:
num = 499
fig, ax = plt.subplots(1,2)
ax[0].imshow(X_train[num])
ax[1].imshow(preds_dec[num])
plt.show()

In [ ]:
# test_sample = np.mean(preds_enc[2],axis=0)
test_sample = np.random.normal(size=512)

test_pred = vae.decoder.predict(test_sample.reshape(1,512))
plt.imshow(test_pred[0])

In [ ]:
vae_test = VAE(encoder, decoder)
vae_test.compile(optimizer=keras.optimizers.Adam())
history = vae_test.fit(X_test, epochs=15, batch_size=32, verbose=1)

In [ ]:
preds_enc_test = vae_test.encoder.predict(X_test)
# preds_enc[0].shape
preds_dec_test = vae_test.decoder.predict(preds_enc_test[2])
# preds_dec.shape

In [ ]:
num = 30
fig, ax = plt.subplots(1,2)
ax[0].imshow(X_test[num])
ax[1].imshow(preds_dec_test[num])
plt.show()

In [ ]:
# test_sample = np.mean(preds_enc_test[2],axis=0)

test_sample = np.random.normal(size=512)
test_pred = vae_test.decoder.predict(test_sample.reshape(1,512))
plt.imshow(test_pred[0])